## Simple Machine Learning Antimalware in Python
I'll try to explain how can you use the **Benign & Malicious PE Files** dataset.

I built the dataset with Python pefile library. The malwares are from [VirusShare](https://virusshare.com/) and the benign software are from a windows server SO.

In [ ]:
#Import the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import pickle as pck
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline

> You can check my [github](https://github.com/amauricio/sklearn-antimalware) to learn how to capture new data from Malwares.

In [ ]:
#Loading dataset from training
data = pd.read_csv('../input/dataset_malwares.csv', sep=',')

#The target is Malware Column {0=Benign, 1=Malware}
X = data.drop(['Name','Malware'], axis=1)
y = data['Malware']

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=101)
X_train.head()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

In [ ]:
X_new = pd.DataFrame(X_scaled, columns=X.columns)
X_new.head()

In [ ]:
skpca = PCA(n_components=55)
X_pca = skpca.fit_transform(X_new)
print('Variance sum : ', skpca.explained_variance_ratio_.cumsum()[-1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
model = RFC(n_estimators=100, random_state=0, 
                         oob_score = True,
                         max_depth = 16, 
                         max_features = 'sqrt')
model.fit(X_pca, y_train)

X_test_scaled = scaler.transform(X_test)
X_test_new = pd.DataFrame(X_test_scaled, columns=X.columns)
X_test_pca = skpca.transform(X_test_new)

y_pred = model.predict(X_test_pca)
print(classification_report(y_test,y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test,y_pred), annot=True, fmt="d", cmap=plt.cm.Blues, cbar=False)

In [ ]:
# from sklearn.externals import joblib
# from sklearn.pipeline import Pipeline
# pipe = Pipeline([('scale', scaler),('pca', skpca), ('clf', model)])
# # jbolib.dumps(pipe, 'my_model')

> Maybe you want save the model to use it later without need to do the training again.

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC()

In [ ]:
model.fit(X_pca,y_train)

In [ ]:
predictions = model.predict(X_test_pca)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
sns.heatmap(confusion_matrix(y_test,predictions), annot=True, fmt="d", cmap=plt.cm.Blues, cbar=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'C':[0.1,1,10,100,1000],'gamma':[1,0.1,0.01,0.001,0.0001]}

In [ ]:
grid = GridSearchCV(SVC(),param_grid,verbose=3)

In [ ]:
grid.fit(X_pca, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test_pca)

In [ ]:
print(classification_report(y_test,grid_predictions))

In [ ]:
sns.heatmap(confusion_matrix(y_test,grid_predictions), annot=True, fmt="d", cmap=plt.cm.Blues, cbar=False)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn.fit(X_pca, y_train)

In [ ]:
pred = knn.predict(X_test_pca)

In [ ]:
sns.heatmap(confusion_matrix(y_test,pred), annot=True, fmt="d", cmap=plt.cm.Blues, cbar=False)

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
error_rate = []
for i in range(1,20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_pca,y_train)
    pred_i = knn.predict(X_test_pca)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,20),error_rate,color='blue',linestyle="--",marker="o", markerfacecolor='red',markersize=10)
plt.title("Error Rate Vs K")
plt.xlabel("K")
plt.xticks(np.arange(0, 20, step=1))
plt.ylabel("Error Rate")

So k=1 is the optimum value of k